# Analysis 2: Foundations of modeling 2

## Recursion: the Fibonacci sequence
#### Recursion
Recursion is a method to address problems where the solution depends on _simpler_ instances of the original problem.  The simplest instances of the problem are called the _base cases_.

We do this by writing a _recursive_ function for this problem: a function that (directly or indirectly) calls _itself_ at some point.
It is important to also treat the base case in a recursive function, for otherwise it would keep calling itself indefinitely.

#### Fibonacci
The archetypical example of a recursively defined sequence in mathematics is the **Fibonacci** sequence.
It is defined as:
$$
F_1 = 1,\quad F_2 = 1,\quad F_n = F_{n-1} + F_{n-2}.
$$
This means that, after the first two terms, every term can be computed by adding the previous 2 terms.
The base cases here are $F_1=1$, $F_2=1$, and the recursion is $F_n = F_{n-1} + F_{n-2}$: 
it expresses $F_n$ using terms of _smaller_ index.

Here's a table containing the first few values:
$$
\begin{array} {|r|r|}\hline n & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 \\ 
\hline F_n & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 \\ \hline  \end{array}
$$

We may even use the recursion to extend the sequence to the left.
What we then get is: 
$$
F_0 = 0,\quad F_{-1} = 1,\quad F_{-2} = -1,\quad F_{-3} = 2, \quad F_{-4} = -3, \quad F_{-5} = 5,\quad\ldots
$$
But we shall mainly focus on terms of positive index (and sometimes 0).

In this notebook we shall explore several Python implementations that can be used to compute $F_n$.
It turns out that this is not as simple as it may look!

This document contains:
- [The obvious implementation](#obvious)
- [Memoization](#memoization)
- [Using a cache decorator](#cache)
- [Two-term recursion](#twoterm)
- [An iterative approach](#iter)
- [Experiment: the tribonacci sequence](#tri)
- [Super-fast Fibonacci recursion (optional)](#superfast)

---

<a id='obvious'></a>
### The obvious implementation

The obvious way to implement the Fibonacci sequence is by directly using the recursive formula in a Python function:

In [ ]:
def fibonacci_obvious(n):
    if n == 1 or n == 2:  # Don't forget about the base cases: there are two of them!
        return 1
    else:
        return fibonacci_obvious(n-1) + fibonacci_obvious(n-2)

Let us verify the first 12 numbers, just to make sure we made no mistakes:

In [ ]:
[fibonacci_obvious(n) for n in range(1, 13)]

However, for a not-too-large value of $n$, this will hang your code execution:

In [ ]:
fibonacci_obvious(100)  # Warning: this will hang!  Interrupt the Kernel to be able to proceed.

The following picture, taken from Wikipedia, explains the problem:

![tree](https://upload.wikimedia.org/wikipedia/commons/a/a3/Call_Tree_for_Fibonacci_Number_F6.svg)

The tree shows all the calls that have to be made to compute $F_6$.  We see that the same number, e.g. $F_3$, is computed over and over again.  Very inefficient!  Luckily, there are ways to resolve this problem.

---
<a id='memoization'></a>
### Memoization

Memoization is an optimization technique where we use the memory (cache) to save results of expensive function calls, with the idea to look them up later, instead of calculating them again. Thus, the faster exeqution is achieved, with the tradoff of using more memory.

Python dictionary is a very convinient structure for such a task. The general idea is to store every fibonacci element that was caclulcated so far in the cache. When a new value is passed to `fibonacci_dict()` function, it will look it up in the dictionary, and if found, just return it. Otherwise, the value will be calculated and added to the cache.

In [ ]:
dict_cache = {}     # note: for simplicity we use cache as global

def fibonacci_dict(n):
    # (part 1) check if n is already in cache and then just return it
    if n in dict_cache:
        return dict_cache[n]
    
    # (part 2) otherwise, use recursion to find the value of F(n)
    if n == 1 or n == 2:
        value = 1
    else:
        value = fibonacci_dict(n-1) + fibonacci_dict(n-2)
    
    # (part 3) for F(n) = value; store it in cache for future use,
    # then return it
    dict_cache[n] = value
    return value

Let us verify the first 12 numbers again.

In [ ]:
[fibonacci_dict(n) for n in range(1, 13)]

Let us inspect what are the contents of our cache.

In [ ]:
print(dict_cache)

And now, try finding out the value for $F_{100}$.

In [ ]:
fibonacci_dict(100)

Impressive, right? Let's check our cache again.

In [ ]:
print(dict_cache)

Try $F_{1000}$.

In [ ]:
fibonacci_dict(1000)

---
<a id='cache'></a>
### Using a cache decorator

One way to tackle the inefficiency of doing the same computation over and over again is using a cache decorator to store the most recently compute values.
This decorator will inspect whether the function has been called with the same argument before, and if so, immediately return the previously computed value.

The Python Standard Library provides such a decorator: `@functools.lru_cache`:

In [ ]:
from functools import lru_cache

@lru_cache()
def fibonacci_cached(n):
    if n == 1 or n == 2:  # Don't forget about the base cases: there are two of them!
        return 1
    else:
        return fibonacci_cached(n-1) + fibonacci_cached(n-2)

Let us try to compute $F_{100}$ using this cached function:

In [ ]:
fibonacci_cached(100)

Better!  Even for larger values, we get an instant answer:

In [ ]:
fibonacci_cached(1000)

A great advantage of using caches is that they require only one line of extra code on top of what you already had.
A downside of using caches, however, is that they may drain your memory.

And they do not solve the problem of reaching the _maximum recursion depth_:
Recursive calls cause the _call stack_ to grow in memory.  If the call stack is full, then we cannot make any further calls, and we will get a `RecursionError` upon doing so.

In [ ]:
fibonacci_cached(1000000)  # This will give a RecursionError

---
<a id='twoterm'></a>
### Two-term recursion

The Fibonacci recursion looks at the last <u>two</u> of the terms of the sequence built up so far.
In view of this, it is a good idea to implement a function `twoterms(n)` that outputs not one, 
but a tuple $(F_{n}, F_{n-1})$ of two consecutive Fibonacci numbers.  To give a couple of examples:

`twoterms(1)` returns `(1, 0)`,\
`twoterms(2)` returns `(1, 1)`,\
`twoterms(3)` returns `(2, 1)`,\
`twoterms(4)` returns `(3, 2)`,\
`twoterms(5)` returns `(5, 3)`,\
`twoterms(6)` returns `(8, 5)`,\
etcetera.

The recursive formula that expresses $(F_n, F_{n-1})$ in terms of $(F_{n-1}, F_{n-2})$ is:
$$
(F_1,\, F_0) = (1,\, 0),\quad (F_{n},\, F_{n-1}) = (F_{n-1}\!+\!F_{n-2},\, F_{n-1}).
$$
And this leads to the following Python implementation:

In [ ]:
def twoterms(n):
    """Return the pair F_n, F_{n-1} of Fibonacci numbers"""
    if n == 1:  # There are two terms in the output, so there is only one base case
        return 1, 0
    else:
        Fn_1, Fn_2 = twoterms(n-1)  # The underscores should be read as minus signs
        Fn = Fn_1 + Fn_2
        return Fn, Fn_1
    

Let's check this for the first 12 terms:

In [ ]:
[twoterms(n) for n in range(1, 13)]

This works.  Let us also define a function that outputs 1 Fibonacci number based on this approach:

In [ ]:
def fibonacci2(n):
    Fn, Fn_1 = twoterms(n)
    return Fn

It works flawlessly for moderately large numbers, without using a memory-consuming cache:

In [ ]:
fibonacci2(1000)

However, it still drains the call stack and causes a recursion error when the input is too large:

In [ ]:
big_num = fibonacci2(1000000)  # Again a RecursionError

---
<a id='iter'></a>
### An iterative approach

Any recursive algorithm can be transformed into an iterative algorithm, i.e. one that uses only iterations and no recursions.
An advantage of this is that it doesn't drain the call stack, and it often dynamically allocates the memory it needs.

Depending on the particular problem, the transfomation is not always easy to perform.  This is why recursion still comes in handy.
However, the two-term approach is perfectly suited for iteration:

In [ ]:
def fibonacci_iterative(n):
    Fm, Fm_1 = 1, 0  # We start with m=1
    for m in range(2, n+1):
        Fm, Fm_1 = Fm + Fm_1, Fm
    return Fm

As always, we check it on a few terms first:

In [ ]:
[fibonacci_iterative(n) for n in range(1, 13)]

Check it on a moderately large value:

In [ ]:
fibonacci_iterative(1000)

Now of course we want to know whether the millionth Fibonacci number can be computed this way:

In [ ]:
big_num = fibonacci_iterative(1000000)  # May take more than 10 seconds (depending on hardware of course)

The number computed has more than 200000 digits.  Display it at your own risk:

In [ ]:
big_num  # Warning: if you execute this code cell, your screen will be flooded with digits!

---
<a id='tri'></a>
### Experiment: the tribonacci sequence

The *tribonacci* sequence is defined using a three-term recursion:
$$
T_0 = 0,\quad T_1 = 0, \quad T_2 = 1, \quad T_{n}=T_{n-1}+T_{n-2}+T_{n-3}.
$$
The first few terms are:
$$
\begin{array} {|r|r|}\hline n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 \\ 
\hline T_n & 0& 0& 1& 1& 2& 4& 7& 13& 24& 44& 81& 149  \\ \hline  \end{array}
$$
- Apply each of the above techniques to implement a function that computes the $n$-th tribonacci number.

---
<a id='superfast'></a>
### Super-fast Fibonacci recursion (optional)

The best way to tackle any programming challenge is to first do your math, then do even more math, 
and only then start typing.
And with a little bit of math, one may find the following identities:
$$
\begin{array}{lcl}
F_{2n-1} &=& F_n^2 + F_{n-1}^2,\\
F_{2n} &=& F_n^2 + 2F_{n-1}{F_n},\\
F_{2n+1} &=& 2F_n^2+2F_{n-1}F_n + F_{n-1}^2.
\end{array}
$$

These identities allow us to speed up a two-term recursion considerably: every call the index will halve!
Let's see how this works in Python:

In [ ]:
def twoterms_superfast(n):
    """Return the pair F_n, F_{n-1} of Fibonacci numbers"""
    if n == 1:  # Base case
        return 1, 0
    else:
        m = n // 2
        Fm, Fm_1 = twoterms_superfast(m)  # Recursive call: m is half of n
        if n%2 == 0:  # n = 2m
            Fn = Fm * (Fm + 2*Fm_1)
            Fn_1 = Fm*Fm + Fm_1*Fm_1
            return Fn, Fn_1
        else:  # n = 2m+1
            Fn = 2*Fm*(Fm + Fm_1) + Fm_1*Fm_1
            Fn_1 = Fm * (Fm + 2*Fm_1)
            return Fn, Fn_1

        
def fibonacci_superfast(n):
    Fn, Fn_1 = twoterms_superfast(n)
    return Fn
        

Let us verify whether this mathematical hocus-pocus does indeed correctly compute $F_n$:

In [ ]:
[fibonacci_superfast(n) for n in range(1, 13)]

Well, this looks good.  Let us now try to compute the millionth term of the Fibonacci sequence:

In [ ]:
big_num = fibonacci_superfast(1000000)

That was done in the blink of an eye!  As mentioned above, the number computed has more than 200000 digits.  In fact, converting this number to a string takes way more time than it took to actually compute it!  Display at your own risk:

In [ ]:
big_num  # Warning: if you execute this code cell, your screen will be flooded with digits!

#### Experiment
- Transform the super-fast Fibonacci code into an iterative algorithm (use the binary representation of n).
- Verify your algorithm on the first 12 Fibonacci numbers
- Also verify that it indeed computes big Fibonacci numbers quickly.